# Определение токсичных комментариев

Интернет-магазин запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Необходимо построить модель со значением метрики качества *F1* не меньше 0.75. 

### Описание данных

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

# 1. Подготовка

In [1]:
import pandas as pd
import re
import numpy as np

from catboost import CatBoostClassifier
import lightgbm as lgb
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score

import nltk
from nltk.corpus import stopwords as nltk_stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

In [3]:
data = pd.read_csv('/datasets/toxic_comments.csv')

In [3]:
data.info()
#df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [4]:
data['toxic'].value_counts()

0    143346
1     16225
Name: toxic, dtype: int64

Наблюдается дисбаланс классов

##### Лемматизация

In [4]:
# Функция лемматизации
l = WordNetLemmatizer()
def lemmatize(text):
    
    word_list = nltk.word_tokenize(text)
    lemm_text = " ".join([l.lemmatize(word) for word in word_list])

    return lemm_text

def clear_text(text):
    text = re.sub(r'[^a-zA-Z0-9]', ' ', text)
    return " ".join(text.split())

In [5]:
data['lemm_text'] = data['text'].apply(lemmatize)

In [6]:
data['lemm_text'] = data['lemm_text'].apply(clear_text)

In [8]:
data.head()

,text,toxic,lemm_text
0,Explanation\nWhy the edits made under my usern...,0,Explanation Why the edits made under my userna...
1,D'aww! He matches this background colour I'm s...,0,D aww He match this background colour I m seem...
2,"Hey man, I'm really not trying to edit war. It...",0,Hey man I m really not trying to edit war It s...
3,"""\nMore\nI can't make any real suggestions on ...",0,More I ca n t make any real suggestion on impr...
4,"You, sir, are my hero. Any chance you remember...",0,You sir are my hero Any chance you remember wh...


##### Разделение на выборки и векторизация текста

In [ ]:
data['lemm_text'] = data['lemm_text'].values.astype('U')

#train, val_test = train_test_split(data, test_size=0.4, random_state=42)
#val, test = train_test_split(val_test, test_size=0.5, random_state=42)

train, test = train_test_split(data, test_size=0.25, random_state=42)

In [ ]:
stopwords = set(nltk_stopwords.words('english'))

count_tf_idf = TfidfVectorizer(stop_words=stopwords)
tf_idf = count_tf_idf.fit(train['lemm_text'])

In [ ]:
features_train = tf_idf.transform(train['lemm_text'])
target_train = train['toxic']
features_test = tf_idf.transform(test['lemm_text'])
target_test = test['toxic']

# 2. Обучение

In [12]:
columns = ['F1-мера']
index = ['Логистическая регрессия', 'LightGBM', 'Константная модель', 'CatBoost', 'Случайный лес']
results = pd.DataFrame(index=index, columns= columns)

##### Логистическая регрессия

In [13]:
model_lr = LogisticRegression(random_state=42, solver='liblinear', class_weight='balanced')
model_lr.fit(features_train, target_train)

LogisticRegression(class_weight='balanced', random_state=42, solver='liblinear')

In [14]:
pred_test = model_lr.predict(features_test)
f1_lr = f1_score(target_test, pred_test)
results.loc['Логистическая регрессия', 'F1-мера'] = f1_lr
f1_lr

0.7453618313985029

##### Случайный лес

In [19]:
model_rfc = RandomForestClassifier(random_state=42, class_weight='balanced')

grid_param = {'n_estimators' : [50 * x for x in range(1, 5)],
              'max_depth' : [5 * x for x in range(1, 5)]}

grid_search = GridSearchCV(model_rfc, grid_param, scoring='f1', n_jobs=-1, cv=3)
grid_search.fit(features_train, target_train)
grid_search.best_params_

{'max_depth': 20, 'n_estimators': 150}

In [20]:
model_rfc = RandomForestClassifier(n_estimators=150, max_depth=20, class_weight='balanced', random_state=42)
model_rfc.fit(features_train, target_train)

RandomForestClassifier(class_weight='balanced', max_depth=20, n_estimators=150,
                       random_state=42)

In [21]:
pred_tes = model_rfc.predict(features_test)
f1_rfc = f1_score(target_test, pred_tes)
results.loc['Случайный лес', 'F1-мера'] = f1_rfc
f1_rfc

0.38692254654102504

##### LightGBM

In [22]:
model_lgbmc = lgb.LGBMClassifier(random_state=42, class_weight='balanced')

grid_param = {
    'max_depth': [30, 20],
    'num_leaves': [70, 50],
    'learning_rate': [0.03],
    'num_boost_round': [700, 800]}

grid_search = GridSearchCV(model_lgbmc, grid_param, scoring='f1', n_jobs=-1, cv=3, verbose=10)
grid_search.fit(features_train, target_train)
grid_search.best_params_

Результат:

{'learning_rate': 0.03,
 'max_depth': 30,
 'num_boost_round': 800,
 'num_leaves': 70}

In [23]:
data_train = lgb.Dataset(features_train, label=target_train)

params =  {'max_depth': 30,
        'learning_rate': 0.03,
        'num_boost_round': 800,
          'num_leaves' : 70}


model_lgbmc =lgb.train(params, data_train)

C:\Void\Soft\Anaconda\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


In [24]:
pred_test = model_lgbmc.predict(features_test)

In [25]:
pred_test = np.where(pred_test > 0.5, 1, 0)

In [26]:
f1_lgbmc = f1_score(target_test, pred_test)
results.loc['LightGBM', 'F1-мера'] = f1_lgbmc
f1_lgbmc

0.7563779086066723

##### CatBoost

In [27]:
#model = CatBoostClassifier(random_seed=42, depth=15, iterations=100, learning_rate=0.3)
#model.fit(features_train, target_train, verbose=10)

In [28]:
#pred = model.predict(features_test)
#f1_cbc = f1_score(target_test, pred_test)
#results.loc['CatBoost', 'F1-мера'] = f1_cbc
#f1_cbc

Обучение занимало слишком много времени.

##### Константная модель

In [ ]:
pred_median = np.ones(target_test.shape)
f1_const = f1_score(target_test, pred_median)
results.loc['Константная модель', 'F1-мера'] = f1_const
f1_const

# 3. Выводы

In [30]:
results.sort_values(by='F1-мера', ascending=False)

,F1-мера
LightGBM,0.756378
Логистическая регрессия,0.745362
Случайный лес,0.386923
Константная модель,0
CatBoost,NaN


+ Логистическая регрессия почти достигла целевого значения метрики (если округлить, как раз 0,75 получится)))
+ После продолжительного подбора параметров LightGBM достиг целевой метрики
+ Случайный лес с задачей не справляется
+ Работа с текстом сложная и ресурсозатратная задача
+ Вычислительные мощности решают